# EPICS V4 Seminar
***

# Aims of course  
* Understand key V4 concepts
* Be able to get started with V4 and know where to find information
* Be able to use V4 command line tools
* Be able to create V4 data structures
* Be able to implement simple V4 server and client examples

# What’s covered:
* What is V4 and what can I do with it?  


* Basic V4 concepts
  * pvData and pvAccess
  * Normative Types
  * pvRequest

* How to get started with V4
  * resources and modules

* V3/V4 interoperability

* pvTools

* Writing a V4 service

* pvData APIs

* Normative Types API

* pvaSrv

* pvDatabase

* pvaClient

# What’s not covered:
* pvAccess APIs
* pvAccess network protocol
* Specific V4 applications
* Java bindings
* Python

# What is EPICS Version 4?
* EPICS is  set of tools, libraries and applications to create distributed control system
* Adds structured data to EPICS

# In EPICS V3 we deal with single values and limited set of data structures.
### PV is a representation of a single value on EPICS V3 host
### Types of these values are from basic set of simple types: 
* DBR_CHAR, DBR_SHORT, DBR_LONG, DBR_FLOAT, DBR_DOUBLE, DBR_STRING, DBR_ENUM

### Channel Access channels are connections to single PV.
### Limited support for structured data:
* DBR_STS types add alarm status and severity
* DBR_TIME adds time stamps to DBR_STS
* DBR_GR adds alarm limits, units and precision to DBR_STS
* DBR_CTRL adds control limits to DBR_GR

# In EPICS V4 there is support for a rich set of data types
## pvData: A system of structured data types
## pvAccess: operations involving pvData structures and network protocol to carry them
* Can get, put, monitor structures. (And do some other things too like RPC)

# What can we do with it?
### V4 can do everything V3 can do (but better)
Can construct pvData structures analogous to DBR types.
For example the equivalent of a DBR_TIME_DOUBLE would be the structure:
```java
NTScalar
	double value
	alarm_t alarm
		int severity
		int status
		string message
	time_t timeStamp
		long secondsPastEpoch 
		int nanoseconds
		int userTag
```
(We’ll cover pvData in more depth later)


In **V4** can create a channel to a structured PV and get a subset of fields equivalent to a DBR type. So for a PV whose principal value is a double we can get the equivalent to DBR_TIME_DOUBLE, which, including values, might look something like
```c++
NTScalar
	double value 8
	alarm_t alarm
		int severity 2
		int status 3
		string message HIHI_ALARM
	time_t timeStamp
		long secondsPastEpoch 1460589140
		int nanoseconds 389605397
		int userTag 0
```

In V4 can go beyond DBR types by asking for combinations not possible in CA. Possible to request arbitrary subsets of these structures
E.g.
time stamp without alarm
Control limits and time stamp at the same time.

In pvAccess operations, on the wire we only need send deltas. So if the value of the structure in the above example is modified:
```java
NTScalar 
	double value 8.1
	alarm_t alarm
		int severity 2
		int status 3
		string message HIHI_ALARM
	time_t timeStamp 
		long secondsPastEpoch 1460589145 
		int nanoseconds 588698520 
		int userTag 0
```
only changed values (in bold) need be sent (plus changed bitset).

Minimum data required to be sent: 
* Changed values: 8.1, 1460589145, 588698520 (8+8+4=20 bytes)
* Changed bitset: {1,7,8} (1+2=3 bytes)  

# V4 fixes a number of problems in V3:
### V4 has better support for Arrays:
* No element_count upper limit (fixed and bounded arrays possible)
* Clear distinction between arrays of size 1 and scalars

### Better support for Strings:  
* Arbitrary size  
* No fixed limit or need for long string workaround

### Much better support for arrays of strings
* Handles arbitrary number of arbitrary lengths strings

### Better support for signed/unsigned types
* Separate types for unsigned and signed and support on the wire for encoding.
    
### 64-bit integers
### Enums not limited to 16 values

# EPICS V4 allows possibility of group operations.
### Possibility of atomic operations on group of PVs. 
* e.g. for PVs X and Y, representing x and y coordinates, can have a channel to group PV and get, for example the values and time stamps of the 2 position PVs:

```java

structure
    NTScalar
		double X 1.1
		time_t timeStamp
			long secondsPastEpoch 1460589140
			int nanoseconds 385960397
			int userTag 0
    NTScalar
        double Y 2.3
        time_t timeStamp
            long secondsPastEpoch 1460589140
            int nanoseconds 385643513
            int userTag 0
```

Note in V4 can also create structures like this:
```java
structure
	structure value
        double X 1.1
        double Y 2.3
	time_t timeStamp
		long secondsPastEpoch 1460589140
		int nanoseconds 385960397
		int userTag 0
```
where we think of a point as being the fundamental object rather than be made up of two coordinates. 

### Complex Data Types

pvData can encode more complex data types like a table:

```java
NTTable
	string[] labels [ value, seconds, nanoseconds, status, severity] 
	structure value
		double[] value [ 1.1, 1.2, 2.0] 
		long[] secondsPastEpoch [1460589140, 1460589141, 1460589142]
		int[] nanoseconds [ 164235768, 164235245, 164235256]
		int[] severity [0, 0, 1]
		int[] status [0, 0, 3]
```

or a request:

```java
NTURI 
	string path archiveService
	string scheme pva 
	structure query
		string entity emittance
		string starttime  1460589140
  		string endtime  1460589143
```


### EPICS V4 Services
pvAccess adds an RPC operation.

Client sends a structure argument and server returns a structure response.

Allows creation of V4 services:

* Archiver service
* Magnet polynomial service
* Snapshot/restore
* Twiss parameters 

### Complex control
Possible to create complex structures representing, for example, a detector, camera driver, file writer or camera plugin.

Can put to subset of fields for control or monitor whole structure.

With RPC can add “methods” and create distributed objects.

### V4 for experiment data: 
Can put experiment data in a structure “PV” and monitor.  
### Examples:
* areaDetector NDArrays
* Neutron data  

For example 1 frame of areaDetector data can be encoded using the structure of the following type:

```java
NTNDArray
    union value
        boolean[] booleanValue
        byte[] byteValue
        short[] shortValue
        int[] intValue
        long[] longValue
        ubyte[] ubyteValue
        ushort[] ushortValue
        uint[] uintValue
        ulong[] ulongValue
        float[] floatValue
        double[] doubleValue
    codec_t codec
        string name
        any parameters
    long compressedSize
    long uncompressedSize
    dimension_t[] dimension
        dimension_t
            int size
            int offset
            int fullSize
            int binning
            boolean reverse
    int uniqueId
    time_t dataTimeStamp
        long secondsPastEpoch
        int nanoseconds
        int userTag
     NTAttribute [] attribute
         NTAttribute
            string name
            any value
            string descriptor
            int sourceType
            string source
```